## Intro

This is a silly project that started with me and my brothers watching overwatch league matches and complaining about how terrible the casters are at filling time. I was looking for a new data science personal project and figured why not try and solve our problem with machine learning. In this notebook, I've written a convolutiuon neurel network to identify gameplay and skip all other parts of the match. It actually works pretty well.

Things I read while doing this

1) http://zulko.github.io/blog/2014/07/04/automatic-soccer-highlights-compilations-with-python/

2) https://www.udemy.com/course/machinelearning/learn/lecture/6761136

3) https://www.toptal.com/machine-learning/machine-learning-video-analysis

4) https://www.analyticsvidhya.com/blog/2018/09/deep-learning-video-classification-python/

5) https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/


In [79]:
import cv2
import math
import pandas as pd
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
import tensorflow as tf
from moviepy.editor import VideoFileClip, concatenate
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Cutting Up Train Videos Into Frames

*NOTE* changed naming convention later on to         filename ="pred/4train_%d.jpg" % count;count+=1 so that it would work properly with the renaming function (so that it would work properly with the flow from directory)


In [221]:
# count = 0
# videoFile = '@Chengdu Hunters vs @Dallas Fuel _ Summer Showdown Tournament _ Day 1-5kVvsQYk2I0.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/1_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

Done!


In [ ]:
# count = 0
# videoFile = 'Losers Semi-Final _ @ATL Reign   vs   @Dallas Fuel _ Summer Showdown Tournament _ Day 2-RKKXYefOz5w.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/2_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

In [ ]:
# count = 0
# videoFile = 'Winners Finals _ @Chengdu Hunters vs  @Shanghai Dragons  _ Summer Showdown Tournament _ Day 2-hyCOml7JcrA.mp4'
# cap = cv2.VideoCapture(videoFile)
# frameRate = cap.get(5)
# while(cap.isOpened()):
#     frameId = cap.get(1)
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="train/3_train%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")

From here, I manually sorted as gameplay vs not gameplay

## Loading In Video to Process

In [6]:
count = 0
videoFile = 'Losers Finals _ @Chengdu Hunters vs @Dallas Fuel  _ Summer Showdown Tournament _ Day 2-dFcIIRQLbpo.mp4'
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)
test_imgID = []
while(cap.isOpened()):
    frameId = cap.get(1)
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="pred/images/4train_%d.jpg" % count;count+=1 # Naming with train is just so its consistent when I examin a new video and add this to train data
        cv2.imwrite(filename, frame)
        test_imgID.append(filename)
cap.release()
print ("Done!")

Done!


Yeah, so I keep running into a sorting issue where 0 comes first, then 1, then 10, then 100. Going to rename my files according to this stackoverflow solution. - https://stackoverflow.com/questions/5417979/batch-rename-sequential-files-by-padding-with-zeroes

In [ ]:
# import os
# path = 'pred/'
# for filename in os.listdir(path):
#     prefix, num = filename[:-4].split('_')
#     num = num.zfill(4)
#     new_filename = prefix + "_" + num + ".png"
#     os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

## Data Preprocessing

In [7]:
image_generator = ImageDataGenerator(rescale = 1./255,
                                   validation_split=.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = image_generator.flow_from_directory(batch_size=32,
                                                 directory='train',
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="training",
                                                 class_mode='binary')
test_set = image_generator.flow_from_directory(batch_size=32,
                                                 directory='train',
                                                 shuffle=True,
                                                 target_size=(64, 64), 
                                                 subset="validation",
                                                 class_mode='binary')

Found 20223 images belonging to 2 classes.
Found 5055 images belonging to 2 classes.


## Building the CNN

In [8]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the CNN

In [9]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [74]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/5
632/632 [==============================] - 213s 337ms/step - loss: 0.0059 - accuracy: 0.9978 - val_loss: 0.0169 - val_accuracy: 0.9953
Epoch 2/5
632/632 [==============================] - 214s 338ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0071 - val_accuracy: 0.9984
Epoch 3/5
632/632 [==============================] - 212s 336ms/step - loss: 0.0065 - accuracy: 0.9980 - val_loss: 0.0118 - val_accuracy: 0.9962
Epoch 4/5
632/632 [==============================] - 206s 326ms/step - loss: 0.0050 - accuracy: 0.9984 - val_loss: 0.0157 - val_accuracy: 0.9955
Epoch 5/5
632/632 [==============================] - 211s 334ms/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.0053 - val_accuracy: 0.9988


### Saving/Loading the model

In [13]:
# cnn_json = cnn.to_json()
# with open("cnn.json","w") as json_file:
#     json_file.write(cnn_json)
# cnn.save_weights("model.h5")

In [170]:
# json_file = open('cnn.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# cnn = tf.keras.models.model_from_json(loaded_model_json)
# cnn.load_weights("model.h5")

## Preprocessing Test Data

going to try and use flow from directory

In [137]:
image_generator2 = ImageDataGenerator(rescale = 1./255)

test_generator = image_generator2.flow_from_directory(batch_size=1,
                                                 directory='pred/',
                                                 target_size=(64, 64), 
                                                 class_mode=None,
                                                 shuffle = False)

Found 6283 images belonging to 1 classes.


## Predictions

In [138]:
test_generator.reset()
pred_probs=cnn.predict_generator(test_generator, steps=len(test_generator), verbose=1)

6283/6283 [==============================] - 45s 7ms/step


In [153]:
pred_probs[:10]

array([[2.9435754e-04],
       [1.3597630e-05],
       [1.0646293e-06],
       [4.8117295e-07],
       [1.0404751e-06],
       [9.3861985e-08],
       [4.2224137e-11],
       [2.0105700e-10],
       [4.8988143e-11],
       [9.5734143e-10]], dtype=float32)

In [140]:
pred_bin=(pred_probs>0.5).astype("int32")

In [142]:
sum(pred_bin)

array([3550])

So, I want some sort of averaging here. Rolling average 

In [117]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

## Turning Predictions Into Something Workable

In [158]:
test_pred = (moving_average(pred_bin.ravel(),30)>.5).astype("int32")

In [159]:
test_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [160]:
count = 0
times = []
for i in range(0,len(test_pred)):
    if test_pred[i] == 1:
        times.append(i)

In [161]:
times[:10]

[301, 302, 303, 304, 305, 306, 307, 308, 309, 310]

In [162]:
count = 0
start_stop = []
for i in range(0,len(times)-1):
    if times[i] == times[i+1]-1:
        count = count + 1
    else:
        start_stop.append((times[i]-count,times[i]))
        count = 0 
start_stop

[(301, 1072),
 (1487, 2471),
 (2508, 2522),
 (2982, 4128),
 (4237, 4274),
 (4754, 4792),
 (4954, 5031),
 (5049, 5142),
 (5175, 5267),
 (5704, 5755),
 (5796, 5831),
 (5865, 5905),
 (5958, 6021)]

So the start and stop times are much more readable, but game play within a couple of minutes shouldn't be in different groups

In [163]:
reduced = [start_stop[0]]
for t in start_stop:
    if t[0] - reduced[-1][1] < 60:
        reduced[-1] = (reduced[-1][0],t[1])
    else:
        reduced.append(t)
reduced

[(301, 1072),
 (1487, 2522),
 (2982, 4128),
 (4237, 4274),
 (4754, 4792),
 (4954, 5267),
 (5704, 6021)]

Looking up above we are doing much better, but we might have a clip with tiny length, so lets make a section of code that clears anything less than 60 seconds (highlight video length)

In [165]:
final_time = []
for item in reduced:
    if item[1]-item[0] > 60:
        final_time.append(item)
final_time

[(301, 1072), (1487, 2522), (2982, 4128), (4954, 5267), (5704, 6021)]

## Writting Out Movie

In [151]:
clip = VideoFileClip(videoFile)
final = concatenate([clip.subclip(max(final_time[i][0]-30,0), min(final_time[i][1]+30,clip.duration)) for i in range(0,len(final_time))])
final.to_videofile('test3.mp4')

Moviepy - Building video test3.mp4.
MoviePy - Writing audio in test3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test3.mp4



Moviepy - Done !
Moviepy - video ready test3.mp4


## Next Steps

~~1) Setup image generator for the predictions. Right now this takes an ungodly amount of ram~~

2) Make a callout for youtube.dl to run everything from this one notebook

3) Add more trainning Data

4) Increase the number of Epochs for the CNN

5) Add in layer graphics

6) Add higher weights for not gameplay time